<center><strong><font size=+3>Calibration demonstration</font></center>
<br><br>
</center>
<center><strong><font size=+2>Matyas Molnar</font><br></strong></center>
<br><center><strong><font size=+1>Astrophysics Group, Cavendish Laboratory, University of Cambridge</font></strong></center>

Gridded plots of redundant visibilities at each stage of the calibration pipeline, inspired by [Josh Dillon's plotting notebook](https://github.com/HERA-Team/hera_sandbox/blob/master/jsd/Data_Inspection/Abscal_Omnical_Demonstration_Plots.ipynb).

In [ ]:
import glob
import os
from copy import deepcopy

import numpy as np
from matplotlib import pyplot as plt

from hera_cal.abscal import post_redcal_abscal_run
from hera_cal.apply_cal import apply_cal
from hera_cal.io import HERAData, HERACal
from hera_cal.redcal import get_reds, redcal_run

from cal_utils import plot_reds, plot_vis, waterfall_column

In [ ]:
%matplotlib inline

In [ ]:
JD = 2458098.43869
pol = 'ee'
bl = (38, 39, 'ee') # baseline examined in this notebook

## Reading the data

In [ ]:
analysis_folder = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/{}'.format(int(JD))
raw_data_fn = 'zen.{}.HH.uvh5'.format(JD)

hera_pkgs = '/users/heramgr/hera_software/'
if not os.path.exists(analysis_folder): # working locally
    analysis_folder = '/Users/matyasmolnar/Downloads/HERA_Data/hc_analysis/sample_calib'
    hera_pkgs = '/Users/matyasmolnar/Downloads/HERA_Data/hera_packages/'

In [ ]:
raw_data_file = os.path.join(analysis_folder, raw_data_fn)


In [ ]:
hd = HERAData(raw_data_file)

In [ ]:
reds = get_reds(hd.antpos, pols=[pol])
red_dict = {bl: bl_group[0] for bl_group in reds for bl in bl_group}
bl_red = red_dict[bl]
print('Baseline {} corresponds to redundant baseline group of type {}'.format(bl, bl_red))

In [ ]:
red_idx = np.where((np.asarray([redg[0] for redg in reds]) == np.asarray(bl_red)).all(axis=1))[0][0]
slct_redg = reds[red_idx][:15] # only choose first 15..
print('Baselines in redundant group {}: {}'.format(bl_red, slct_redg))

In [ ]:
red_data, flags, nsamples = hd.read(bls=slct_redg) # only load redundant group

## Raw visibilities

In [ ]:
plot_reds(red_data, slct_redg, 'Phase of Uncalibrated Visibilities for Baselines Redundant to {}'.format(bl_red))

## Bad antennas

In [ ]:
bad_ants_file = os.path.join(hera_pkgs, 'hera_opm/pipelines/h1c/idr2/v2/bad_ants/{}.txt'.format(int(JD)))
bad_ants = np.loadtxt(bad_ants_file).astype(int)
print('Bad antennas for JD {} are: {}'.format(int(JD), list(bad_ants)))

In [ ]:
red_data_ex_ant = deepcopy(red_data)
for (i, j, pol) in red_data_ex_ant.keys():
    if i in bad_ants or j in bad_ants:
        red_data_ex_ant[(i, j, pol)] = np.zeros_like(red_data_ex_ant[(i, j, pol)])

In [ ]:
plot_reds(red_data_ex_ant, slct_redg, 'Phase of Uncalibrated Visibilities for Baselines Redundant to {}'.format(bl_red))

## Firstcal

## Redcal

 ## Abscal

## LST-binned